## Blatt 06

Abgabe von: __Pape, Kusurmann und Becker__

## ___Damit der Code durchläuft muss die `NeutrinoM.hdf5` Datei in dem Ordner des Notebooks vorliegen.___

### Aufgabe 17: _Datenaufbereitung_

### a.)
__Frage:__ Wie sollten nicht-numerische Datentypen wie beispielsweise Strings vor der Analyse
behandelt werden müssen?

__Antwort:__ Die zu analysierenden Daten müssen in eine statistisch auswertbare Darstellung gebracht werden. Dafür könnten beispielsweise Attribute wie die Stringlänge ausgewertet werden. Angepasst an das Problem muss Vorwissen (_educatet guesses_) in die ausgewählten Attribute gesteckt werden.

### b.)
__Frage:__ Kann es hilfreich sein Attribute zu normieren? Wenn ja, wieso?

__Antwort:__ Eine Normierung der Daten kann die Zuordnung eines neuen Elements zu einer aus den Attributen abgeleiteten Gruppe erleichtern.
Betrachtet man das Beispiel aus der Vorlesung mit der Zuordnung von Texten zu Atheisten und Gläubigen.
Wir nehmen an, dass Attribute gefunden wurden, die beide Klassen gut voneinander trennt. Falls unser Testdatensatz aus sehr vielen Beiträgen von Gläubigen, aber nur wenigen Beiträgen von Atheisten besteht, ist die Zuordnung eines neuen Textes zu einer der beiden Gruppen leichter, wenn mit normierten Attributen gearbeitet wird. Darüberhinaus ist es z.B bei dem KNN-Algorithmus sinnvoll Attribute zu normieren, da so das Abstandsmaß die Attribute gleich gewichtet (vgl. `Aufgabe 15 a.)`.

### c.)
__Frage:__ Wie kann mit Lücken in den Daten oder NaNs und Infs verfahren werden?

__Antwort:__ Lücken, NaNs oder Infs werden aus dem Datensatz gelöscht.

### d.)
__Frage:__ Was ist beim Zusammenführen von Datensätzen zu beachten?

__Antwort:__ Die Datensätze müssen die selben Attribute besitzen. Falls man normierte Datensätze zusammenführt, muss nach dem Zusammenführen der neue Datensatz erneut normieren.
Die Daten der verschiedenen Datensätze sollten gelabelt sein, damit man diese im Nachhinein noch unterscheiden kann.

### e.)
__Frage:__ Welche Attribute sollten vor dem Trainieren des Klassifizierers aus dem Datensatz
entfernt werden? Wie kann dabei eine Reduktion redundanter Informationen erreicht werden? Was muss speziell bei simulationsbasierten Methoden berücksichtigt
werden?

__Antwort:__ Gleichmäßig verteilte Datenpunkte eines Attributs sollten vernachlässigt werden, da sie keinen Informationgewinn enthalten.
Mit Hilfe der `PCA` kann die Dimensionalität des Datensatzes verringert werden, da eine Vielzahl statistischer Variablen durch eine geringere Zahl möglichst aussagekräftiger Linearkombination genähert wird.
Speziell mit simulationsbasierten Methoden muss berücksichtigt werden, dass nicht die Monte-Carlo Wahrheit in den Datensatz einfließt.

### Aufgabe 15: _k-NN Klassifikation_

#### a.)

__Frage:__ Worauf müssen Sie bei einem k-NN-Algorithmus achten, wenn die Attribute sich
stark in ihren Größenordnungen unterscheiden?

__Antwort:__ Der Radius nach dem nächste Nachbarn klassifiziert werden muss auf die Größenskala der Attribute angepasst werden.

#### b.)

__Frage:__ Warum bezeichnet man den k-NN als sogenannten _lazy learner_? Wie sind die
Laufzeiten für Lern- und Anwendungs-Phase? Wie sind sie im Vergleich zu anderen
Algorithmen wie bspw. einer SVM?

__Antwort:__ Der k-NN Klassifikator wird als _leazy learner_ bezeichnet, da die Modellbildung erst zur Zeit der Abfrage geschieht (Quelle: https://de.wikipedia.org/wiki/Lazy_learning).
Die Laufzeit der Anwendungszeit ist viel größer als die Lerzeit, da für alle zu testenden Datenpunkte die Abfrage der nächsten Nachbarn gemacht werden muss.
Der SVM-Algorithmus ist für hochdimensionale Datensätze schneller, als der k-NN Klassifikator
Allgemein ist die Laufzeit der Anwendungsphase für _leazy learner_ länger als für sogenannte _eager learner_.

#### c.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Code aus der beiliegenden `class_structure.py` mit eingefügtem Code:

In [ ]:
class KNN:
    '''KNN Classifier.

    Attributes
    ----------
    k : int
        Number of neighbors to consider.
    '''
    def __init__(self, k):
        '''Initialization.
        Parameters are stored as member variables/attributes.
        
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        '''
        self.k = k
        '''
        Store training data X and label y in class object.
        '''
        self.training_data_X = None
        self.training_data_y = None

    def fit(self, X, y):
        '''Fit routine.
        Training data is stored within object.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Training data.
        y : numpy.array shape=(n_samples)
            Training labels.
        '''
        self.training_data_X = X
        self.training_data_y = y

    def predict(self, X):
        '''Prediction routine.
        Predict class association of each sample of X.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Data to classify.
        
        Returns
        -------
        prediction : numpy.array, shape=(n_samples)
            Predictions, containing the predicted label of each sample.
        -------
        dist findet den Abstand zwischen jedem Datenpunkt
        aus dem Trainingsdatensatz und einem Datenpunkt
        aus dem Testdatansatz. Dies wird iterativ für
        jeden Datenpunkt aus dem Testdatensatz bestimmt.
        '''
        
        dist = [np.sqrt((j - self.training_data_X[0])**2
                        + (k - self.training_data_X[1])**2
                        + (l - self.training_data_X[2])**2)
                for j, k, l in zip(X[0], X[1], X[2])]
        
        '''
        label_trainig_data enthält die label der 10 nächsten
        Nachbarn des Trainigsdaten zu jedem Testdatenpunkt 
        '''
        label_training_data = np.argsort(dist)[:, :self.k]
        
        label_test_data = []

        for n in range(len(X[0])):
            # Abspeichern der 10 label der nächsten Nachbarn für den n-ten Datenpunkt.
            label_buffer = self.training_data_y[label_training_data[n, :]]
            
            #  Bestimmen, ob mehr Nachbarn Untergrund oder Signal sind.
            if len(label_buffer[label_buffer==0]) >= len(label_buffer[label_buffer==1]):
                label_test_data.append(0)
            elif len(label_buffer[label_buffer==1]) > len(label_buffer[label_buffer==0]):
                label_test_data.append(1)
        return np.array(label_test_data)

### d.)

Einlesen der Daten aud der `NeutrinoMC.hdf5` Datei.
Auf die Signale wird die Acceptance Mask angewendet, sodass nur Daten verwendet werden, die vom Detektor registriert wurden.

$k = 10$ sollte nach Aufgabenstellung gewählt werden. Da $k$ gerade ist kann es vorkommen, dass genauso viele NN Signal wie Untergrund sind. Falls dieser Fall eintrifft ist der Algorithmus so eingestellt, dass er diesen Testpunkt als Untergrund erkennt, damit die Reinheit möglichst groß gehalten wird. Allgemein ist es besser ein ungerades $k$ zu wählen, damit dieses Problem umgangen wird.

In [ ]:
signal = pd.read_hdf('NeutrinoMC.hdf5', 'Signal')

signal = signal[signal.AcceptanceMask]

background = pd.read_hdf('NeutrinoMC.hdf5', 'Background')

k = KNN(10)

Erzeugen von 5000 zufällig aus dem `.hdf5` gewählten Trainingsdatenpunkten.

In [ ]:
N_training = 5000

training_signal = signal.sample(N_training)

training_background = background.sample(N_training)

training_data = pd.DataFrame({
    'x' : training_signal['x'].append(training_background['x']),
    'y' : training_signal['y'].append(training_background['y']),
    'NumberOfHits' : training_signal['NumberOfHits'].append(
        training_background['NumberOfHits'])
})

Die Label wurden so gewählt, dass das Label 1 ein Signal und 0 ein Untergrundereignis repräsentiert.

Die `fit`-Methode wird aufgerufen, damit die Trainingsdaten in das Klassenelement $k$ gespeichert werden.

In [ ]:
training_label = np.append(np.ones(N_training), np.zeros(N_training))
k.fit(X=[training_data['x'],
         training_data['y'],
         training_data['NumberOfHits']],
      y=training_label)

Erzeugen von 10.000 Testsignalen und 20.000 Testuntergrundsignalen.

In [ ]:
N_test = 10000

test_signal = signal.sample(N_test)

test_background = background.sample(2 * N_test)

test_data = pd.DataFrame({
    'x' : test_signal['x'].append(test_background['x']),
    'y' : test_signal['y'].append(test_background['y']),
    'NumberOfHits' : test_signal['NumberOfHits'].append(
        test_background['NumberOfHits'])
})

`predict`-Methode bestimmt die mit dem KNN-Algorithmus ermittelten Label der Testdaten.

In [ ]:
test_label = k.predict(X=[test_data['x'],
                          test_data['y'],
                          test_data['NumberOfHits']])

Testen welche Signale und welche Hintergrundereignisse richtig zugeordnet wurden.

In [ ]:
signal_test = np.array([test_data['x'][test_label == 1],
                        test_data['y'][test_label == 1],
                        test_data['NumberOfHits'][test_label == 1]])
background_test = np.array([test_data['x'][test_label == 0],
                            test_data['y'][test_label == 0],
                            test_data['NumberOfHits'][test_label == 0]])

Bestimmen der Effizienz, Reinheit und Signifikanz.
Die Signifikanz wird berechnet mit
$$S = \frac{tp}{\sqrt{tp + fp}}.$$

`true_test_label` enthält die richtigen Label der Testdaten, da diese aus der ausgelesenen `.hdf5` Datei bekannt waren.

In [ ]:
true_test_label = np.append(np.ones(N_test), np.zeros(2 * N_test))

tp = len(test_label[(true_test_label==1) & (test_label==1)])
fn = len(test_label[(true_test_label==1) & (test_label==0)])
fp = len(test_label[(true_test_label==0) & (test_label==1)])

Bestimmen der `Effizienz`, der `Reinheit` und der `Signifikanz`.
Könnt ihr in der Übung nochmal erklären was die `Signifikanz` über der verwendeten Algorithmus oder die Daten aussagt?

In der Vorlesung wurde der Begriff `Signifikanz` nicht eingeführt. Wir haben jetzt die Definition von `Blatt 04` verwendet.

In [ ]:
effizienz = tp / (tp + fn)
reinheit = tp / (tp + fp)
signifikanz = tp / np.sqrt(tp + fp)

print(f'Effizienz = {effizienz}\nReinheit = {reinheit}\nSignifikanz = {signifikanz}')

#### e.)
Was ändert sich, wenn Sie log10(AnzahlHits) statt AnzahlHits nutzen?

Erstellen eines DataFrames mit logarithmierten dritten Attribut.
Im Verlauf des Codes wird analog zu Aufgabenteil `d.)` vorgegangen.

In [ ]:
training_data_log_hits = pd.DataFrame({
    'x' : training_signal['x'].append(training_background['x']),
    'y' : training_signal['y'].append(training_background['y']),
    'log10(NumberOfHits)' : np.log10(training_signal['NumberOfHits']).append(
        np.log10(training_background['NumberOfHits']))
})

In [ ]:
k.fit(X=[training_data_log_hits['x'],
         training_data_log_hits['y'],
         training_data_log_hits['log10(NumberOfHits)']],
      y=training_label)

In [ ]:
test_data_log_hits = pd.DataFrame({
    'x' : test_signal['x'].append(test_background['x']),
    'y' : test_signal['y'].append(test_background['y']),
    'log10(NumberOfHits)' : np.log10(test_signal['NumberOfHits']).append(
        np.log10(test_background['NumberOfHits']))
})

In [ ]:
test_label_log_hits = k.predict(X=[test_data_log_hits['x'],
                                   test_data_log_hits['y'],
                                   test_data_log_hits['log10(NumberOfHits)']])

In [ ]:
signal_test_log_hits = np.array([test_data_log_hits['x'][test_label_log_hits == 1],
                                 test_data_log_hits['y'][test_label_log_hits == 1],
                                 test_data_log_hits['log10(NumberOfHits)'][test_label_log_hits == 1]])
background_test_log_hits = np.array([test_data_log_hits['x'][test_label_log_hits == 0],
                                     test_data_log_hits['y'][test_label_log_hits == 0],
                                     test_data_log_hits['log10(NumberOfHits)'][test_label_log_hits == 0]])

In [ ]:
tp_log = len(true_test_label[(true_test_label==1) & (test_label_log_hits==1)])
fn_log = len(true_test_label[(true_test_label==1) & (test_label_log_hits==0)])
fp_log = len(true_test_label[(true_test_label==0) & (test_label_log_hits==1)])

In [ ]:
effizienz_log = tp_log / (tp_log + fn_log)
reinheit_log = tp_log / (tp_log + fp_log)
signifikanz_log = tp_log / np.sqrt(tp_log + fp_log)

print(f'Effizienz = {effizienz_log}\nReinheit = {reinheit_log}\nSignifikanz = {signifikanz_log}')

Alle drei Werte sind größer als bei dem Test ohne logarithmiertes drittes Attribut.

In den Plotts am Ende der Aufgabe sieht man auch, dass das Signal deutlich weniger _verschmiert_ ist als
in dem Plot der vorigen Daten aus `d.)`.

#### f.)
Was ändert sich, wenn Sie k = 20 statt k = 10 verwenden?

In [ ]:
k_20 = KNN(20)
k_20.fit(X=[training_data['x'],
            training_data['y'],
            training_data['NumberOfHits']],
         y=training_label)

In [ ]:
test_label_20 = k_20.predict(X=[test_data['x'],
                                test_data['y'],
                                test_data['NumberOfHits']])

In [ ]:
signal_test_20 = np.array([test_data['x'][test_label_20 == 1],
                           test_data['y'][test_label_20 == 1],
                           test_data['NumberOfHits'][test_label_20 == 1]])
background_test_20 = np.array([test_data['x'][test_label_20 == 0],
                               test_data['y'][test_label_20 == 0],
                               test_data['NumberOfHits'][test_label_20 == 0]])

In [ ]:
tp_20 = len(true_test_label[(true_test_label==1) & (test_label_20==1)])
fn_20 = len(true_test_label[(true_test_label==1) & (test_label_20==0)])
fp_20 = len(true_test_label[(true_test_label==0) & (test_label_20==1)])

In [ ]:
effizienz_20 = tp_20 / (tp_20 + fn_20)
reinheit_20 = tp_20 / (tp_20 + fp_20)
signifikanz_20 = tp_20 / np.sqrt(tp_20 + fp_20)

print(f'Effizienz = {effizienz_20}\nReinheit = {reinheit_20}\nSignifikanz = {signifikanz_20}')

Die drei Werte sind alle schlechter als bei den Werten aus Aufgabenteil `d.)`.
Dies sieht man auch in dem unteren Plot zu Aufgabenteil `f.)`, da das Signal _verschmierter_ in dem Untergrund ist als bei den anderen Aufgabenteilen.

Plotten der Ergebnisse:

In [ ]:
plt.figure(figsize=[15,15])

plt.subplot(2,2,1)

plt.scatter(test_data['x'][N_test:], test_data['y'][N_test:], s=5, alpha=0.3, label='True Background', color='C1')
plt.scatter(test_data['x'][:N_test], test_data['y'][:N_test], s=5, alpha=0.5, label='True Signal', color='C0')

plt.legend(fontsize='large', loc='upper left')
plt.xlabel(r'$x$-Achse')
plt.ylabel(r'$y$-Achse')

##########################################
plt.subplot(2,2,2)

plt.scatter(background_test[0], background_test[1], s=5, alpha=0.2, color='C1', label='Test Background')
plt.scatter(signal_test[0], signal_test[1], s=5, alpha=0.4, color='C2', label='Test Signal')

plt.xlabel(r'$x$-Achse')
plt.ylabel(r'$y$-Achse')
plt.legend(fontsize='large')

##########################################
plt.subplot(2,2,3)

plt.scatter(background_test_log_hits[0], background_test_log_hits[1], s=5, alpha=0.5, color='C1', label=r'Test Background$_{log10}$')
plt.scatter(test_data['x'][:N_test], test_data['y'][:N_test], s=5, alpha=0.5, color='C0', label=r'Signal$_{log10}$')
plt.scatter(signal_test_log_hits[0], signal_test_log_hits[1], s=5, alpha=0.5, color='C2', label=r'Test Signal$_{log10}$')

plt.legend(fontsize='large')
plt.xlabel(r'$x$-Achse')
plt.ylabel(r'$y$-Achse')

##########################################
plt.subplot(2,2,4)

plt.scatter(background_test_20[0], background_test_20[1], s=5, alpha=0.5, color='C1', label='Test Background k = 20')
plt.scatter(signal_test_20[0], signal_test_20[1], s=5, alpha=0.5, color='C2', label='Test Signal k = 20')

plt.legend(fontsize='large')
plt.xlabel(r'$x$-Achse')
plt.ylabel(r'$y$-Achse')

plt.show()

Der Plot links oben zeigt das wahre Signal aus der `.hdf5` Datei an.

Der Plot rechts oben zeigt das rekonstruierte Signal aus Aufgabenteil `d.)` an.

Der Plot unten links zeigt die Ergebnisse von Aufgabenteil `e.)`. Im blau sind die wahren Testsignale mit logarithmierter Hit Anzahl dargestellt.

Der Plot unten rechts zeigt das mit $k = 20$ rekonstruierte Signal aus Aufgabenteil `f.)`.

### Aufgabe 16: _Binärer Entscheidungsbaum: Die erste Entscheidung_

Angaben vom Übungsblatt:

In [ ]:
temp = [29, 26, 28, 21, 20, 18, 17, 22, 20, 24, 24, 22, 27, 22]
vorhersage = [2, 2, 1, 0, 0, 0, 1, 2, 2, 0, 2, 1, 1, 0]
luftfeuchtigkeit = [80, 90, 70, 90, 80, 70, 65, 90, 70, 80, 70, 90, 75, 80]
wind = [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1]
fußball = [0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]

In [ ]:
df_test = pd.DataFrame()
df_test['temp'] = temp
df_test['vor'] = vorhersage
df_test['luftf'] = luftfeuchtigkeit
df_test['wind'] = wind
df_test['fußball'] = fußball

In [ ]:
def entropie(S, S_c):
    return - S_c / S * np.log2(S_c / S)

def gain(S, S_c):
    return (entropie(len(S), len(S[S == True]))
            + entropie(len(S), len(S[S == False]))
            - sum(S_c == True) / len(S) *
            (entropie(sum(S_c == True), sum((S_c == True) & (S == True)))
            + entropie(sum(S_c == True), sum((S_c == True) & (S == False))))
            - sum(S_c == False) / len(S) *
            (entropie(sum(S_c == False), sum((S_c == False) & (S == False)))
            + entropie(sum(S_c == False), sum((S_c == False) & (S == True)))))

#### a.)
Bestimmen der Entropie der Wurzel.

Die Entropie wird nach der folgenden Formel bestimmt:

$$H(S) = - \sum_{c=1}^{|C|}p_c\log_2(p_c).$$

Dabei ist $S$ der Datensatz, $|C|$ die Anzahl der Kategorien und $p_c$ der Anteil er Instanz in S, die Kategorie c angehört.
(vgl. http://www.coli.uni-saarland.de/courses/mathe3/SS12/Vorlesungen/decisiontree.pdf, Folie 16)

In [ ]:
S_root = (entropie(len(df_test.fußball), len(df_test.fußball[df_test.fußball == True]))
          + entropie(len(df_test.fußball), len(df_test.fußball[df_test.fußball == False])))
print(f'Entropie der Wurzel: {S_root:.5}')

#### b.)
Ermitteln des Informationszuwachses durch das Attribut `Wind`.

Die Funktion `gain` wird wie folgt konstruiert:

$$gain(S,A) = H(S) - \sum_{v\in A}\frac{|S_v|}{|S|} H(S_v).$$

(vgl. http://www.coli.uni-saarland.de/courses/mathe3/SS12/Vorlesungen/decisiontree.pdf Folie 18)

In [ ]:
gain_wind = gain(df_test.fußball, df_test.wind)

print(f'Informationsentropie des Attributes Wind: {gain_wind:.5}')

#### c.) 
Ermitteln des Informationszuwachses durch verschiedene Schnitte der übrigen Attribute.

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=2, criterion='entropy')
clf.fit(np.array(wind).reshape(-1, 1), fußball)
tree.export_graphviz(clf,out_file='test.dot') 

In [ ]:
def H(end_state, test_state):
    n = sum(end_state)
    H = []
    for i in test_state:
        H_i = []
        for j in end_state:
            H_i.append( -j/n *  np.log2(j/i))
           
        H.append(sum(H_i))
    return sum(H)

In [ ]:
# True Fußball
n_p = sum(df_test.fußball)
# False Fußball
n_pn = sum(df_test.fußball == False)

# negativ weather forecast
n_s = sum(df_test.vor == 2)
# neutral weather forecast
n_n = sum(df_test.vor == 1)
# positiv weather forecast
n_g = sum(df_test.vor == 0)

In [ ]:
H_forecast = H([n_p, n_pn], [n_s, n_n, n_g])

information_gain_forecast = S_root - H_forecast

print('Der Informationsgehalt bei einem Schnitt',
       f'auf dem Attribut Wettervorhersage durchgeführt \n wird, liegt bei {information_gain_forecast:.2}')

In [ ]:
# Luftfeuchtigkeit x >= 90 %
n_90 = sum(df_test.luftf >= 90)
# Luftfeuchtigkeit 90 > x >= 80 %
n_80 = sum((df_test.luftf < 90) & (df_test.luftf >= 80))
# Luftfeuchtigkeit 80 > x >= 70%
n_70 = sum((df_test.luftf < 80) & (df_test.luftf >= 70))
# Luftfeuchtigkeit 70 >x >= 60%
n_60 = sum((df_test.luftf < 70) & (df_test.luftf >= 60))

H_humanity = H([n_p, n_pn], [n_90, n_80, n_70, n_60])

inforamtion_gain_humanity = S_root - H_humanity

print('Der Informationsgehalt bei einem Schnitt',
       f'auf dem Attribut Luftfeuchtigkeit durchgeführt \n wird, liegt bei {inforamtion_gain_humanity:.2}')

#### d.)

Die Wettervorhersage.

### Aufgabe 18: _Naive Bayes: Fußball_


#### a.)

Die bedingte Wahrscheinlichkeit ist definiert als

$$
P(A|B) = \frac{P(A\cap B)}{P(B)}.
$$
Es gilt 

$$
P(A|B)P(B) = P(A\cap B) = P(B\cap A) = P(B|A)P(A)\quad \mathrm{q.e.d.}
$$
Damit ist der __Satz von Bayes__ bewiesen.

#### b.)
Wir möchten wissen: Wie hoch die Wahrscheinlichkeit ist, dass wir, bei den gegebenen Wetterbedingungen $W$, Fußball spielen $F$ gehen.
Präzise erfasst wird diese Fragestellung durch:
$$
P(F|W) = \frac{P(F)}{P(W)}P(W|F).
$$
Hierbei gibt $P(F)$ die Wahrscheinlichkeit an das wir überhaupt Fußball spielen gehen und $P(W)$ die Wahrschelichkeit die angegebene Wetterkonfiguration zuerhalten.

Wenn $n$ die Gesamtzahl unserer Messungen ist und $n_F$ angibt, wie oft wir Fußball spielen waren. 
So gilt:
$$
P(F)= \frac{n_F}{n} = \frac{9}{14}
$$
Wenn $n_{windig}$ die Anzahl die Tage ist an den der Wind stark war, $n_{high-humidity}$ die Anzahl der Tage mit einer hohen Luftfeutigkeit, $n_{cold}$ die Anzahl der Tage wo es kalt ist, $n_{sunny}$ die Tage mit sonnigen Aussichten, dann kann $P(W)$ geschrieben werden als:
$$
P(W)=\frac{n_{windy}\cdot n_{high-humidity} \cdot n_{cold} \cdot n_{sunny}} {n^4} = \frac{6\cdot7\cdot6\cdot3}{14^4}=\frac{756}{14^4}
$$
Nach Voraussetzung gilt 
$$
P(W|F)=\Pi_i P(x_i|F).
$$
P(W|F) gibt an wie Wahrscheinlichkeit ist dieses Wetter zu finden, wenn man Fußball spielen geht.

$$
P(W|F)=P(windy|F)\cdot P(high-humidity|F)\cdot P(cold|F)\cdot P(sunny|F) = \frac{3}{9}\cdot\frac{3}{9}\cdot\frac{3}{9}\cdot\frac{2}{9}=\frac{54}{6561}=\frac{2}{243}
$$
Somit ist $P(F|W)$, also die Wahrscheinlichkeit Fußball spielen zu gehen, gleich

$$
P(F|W) =  \frac{P(F)}{P(W)}P(W|F) \approx 0.27 \rightarrow 27\%
$$

#### c.)
Das Problem hier ist, dass wir noch nie Fußball spielen gegenagen sind als es heiß war, somit können wir keine wirkliche Aussage über $P(F|W)$ treffen. Stattdessen können wir berechnen, wie wahrscheinlich es ist das wir nicht Fußball spielen $NF$ gehen, bei der gegebenen Wetterkonfiguration. 
$$
P(NF|W) = \frac{P(NF)}{P(W)}P(W|NF)
$$
mit 
$$\begin{aligned}
P(NF)&= \frac{n_{NF}}{n} = \frac{5}{14},\\
P(W)&=\frac{n_{not-windy}\cdot n_{high-humidity} \cdot n_{hot} \cdot n_{sunny}}{n^4} = \frac{8\cdot7\cdot1\cdot3}{14^4}=\frac{168}{14^4}, \\
P(W|NF)&=\Pi_i P(x_i|NF) = P(not-windy|NF)\cdot P(high-humidity|NF)\cdot P(hot|NF)\cdot P(sunny|NF) = \frac{2\cdot4\cdot1\cdot1}{5^4}=\frac{8}{5^4}
\end{aligned}$$
ergibt sich
 
$$
P(NF|W) = \frac{P(NF)}{P(W)} P(W|NF) \approx 1 \quad \text{Hier kommt ein Wert größer knapp größer 1 raus, dass kann eigentlich nicht so sein!} 
$$
Dann ist 
$$
P(F|W) = 1 - P(NF|W) \approx 0
$$